In [167]:
import os
from datetime import timedelta, datetime
import talib
import pandas as pd
from sqlalchemy import create_engine
import dotenv

In [168]:
dotenv.load_dotenv()
SQLALCHEMY_DATABASE_URI = os.getenv('SQLALCHEMY_DATABASE_URI')

In [169]:
db_engine = create_engine(
    SQLALCHEMY_DATABASE_URI, 
    pool_size=10, 
    max_overflow=20,
    )

In [170]:
request_dict = {
    'symbol': 'HDFCBANK',
    'dates': {
        'start': '2020-01-01',
        'end': '2020-06-01'
    }
}

In [171]:
df = pd.read_sql('''
    SELECT *
    FROM stock_daily_data
    WHERE symbol = '{}'
    AND   TIMESTAMP BETWEEN '{}' AND '{}'
    ORDER BY TIMESTAMP
    LIMIT 1000;
    '''.format(
            request_dict['symbol'],
            datetime.strptime(request_dict['dates']['start'], '%Y-%m-%d') - timedelta(days=500),
            request_dict['dates']['end'],
        ), db_engine, index_col=['timestamp']);

In [172]:
'''
# Generate CSV
df.to_csv('CSV/{}_{}_{}.csv'.format(
    request_dict['symbol'],
    request_dict['dates']['start'],
    request_dict['dates']['end'],
))
'''
# Preview
df_extended = df.copy(deep=True)
df_extended

,Open,High,Low,Close,Volume,symbol,updated_at
timestamp,,,,,,,
2018-08-20,1037.5,1053.80,1037.5000,1048.1000,3905568,HDFCBANK,2020-04-17 03:06:36
2018-08-21,1049.0,1056.00,1046.3199,1048.0300,3888160,HDFCBANK,2020-04-17 03:06:36
2018-08-23,1052.5,1053.50,1038.4301,1039.9500,6287276,HDFCBANK,2020-04-17 03:06:36
2018-08-24,1035.5,1046.80,1033.0300,1034.9301,4857314,HDFCBANK,2020-04-17 03:06:36
2018-08-27,1038.5,1044.25,1035.1200,1040.2200,6287394,HDFCBANK,2020-04-17 03:06:36
...,...,...,...,...,...,...,...
2020-05-26,857.0,871.75,848.7500,852.4000,19002589,HDFCBANK,2020-08-06 06:13:25
2020-05-27,859.9,909.80,857.1500,903.6500,28013227,HDFCBANK,2020-08-06 06:13:25
2020-05-28,920.0,950.00,913.9500,945.2500,37744926,HDFCBANK,2020-08-06 06:13:25


In [173]:
def prepare_data(symbol_dataframe):
    symbol_dataframe['close_prev'] = symbol_dataframe['Close'].shift(1)

    for n_d in [5, 25, 50, 200]:
        symbol_dataframe['EMA{}'.format(n_d)] = get_ema(
            symbol_dataframe, 
            n_days=n_d
            )
        symbol_dataframe['EMA{}_prev'.format(n_d)] = \
            symbol_dataframe['EMA{}'.format(n_d)].shift(1)
    symbol_dataframe['rsi'] = get_rsi(symbol_dataframe, n_days=14)
    symbol_dataframe['sar'] = get_sar(symbol_dataframe, acceleration=0.02, maximum=0.2)
    symbol_dataframe['sar_prev'] = symbol_dataframe['sar'].shift(1)
    
    macd, macdsignal, macdhist = get_macs(
        symbol_dataframe,
        fastperiod=12, 
        fastmatype=0, 
        slowperiod=26, 
        slowmatype=0, 
        signalperiod=9, 
        signalmatype=0,
    )
    symbol_dataframe['macd'] = macd
    symbol_dataframe['macd_prev'] = symbol_dataframe['macd'].shift(1)
    symbol_dataframe['macdsignal'] = macdsignal
    symbol_dataframe['macdsignal_prev'] = symbol_dataframe['macdsignal'].shift(1)
    symbol_dataframe['macdhist'] = macdhist
    symbol_dataframe['macdhist_prev'] = symbol_dataframe['macdhist'].shift(1)
    
    slowk, slowd = get_stoch(
        symbol_dataframe,
        fastk_period=14, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0
    )
    symbol_dataframe['slowk'] = slowk
    symbol_dataframe['slowk_prev'] = symbol_dataframe['slowk'].shift(1)
    symbol_dataframe['slowd'] = slowk
    symbol_dataframe['slowd_prev'] = symbol_dataframe['slowd'].shift(1)

    upperband, middleband, lowerband = get_bbands(
        symbol_dataframe, 
        timeperiod=5, nbdevup=2, nbdevdn=2, matype=0,
        )
    symbol_dataframe['upperband'] = upperband
    symbol_dataframe['middleband'] = middleband
    symbol_dataframe['lowerband'] = lowerband

    symbol_dataframe['obv'] = get_obv(symbol_dataframe)

    symbol_dataframe['adx'] = get_adx(symbol_dataframe, timeperiod=14)
    
    symbol_dataframe['atr'] = get_atr(symbol_dataframe, timeperiod=14)

    # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    # print(symbol_dataframe)
    return symbol_dataframe

def get_ema(symbol_dataframe, n_days):
    '''
    calculate Exponential Moving Average
    '''
    return talib.EMA(symbol_dataframe.Close, timeperiod=n_days)

def get_rsi(symbol_dataframe, n_days):
    '''
    calculate Exponential Moving Average
    '''
    return talib.RSI(symbol_dataframe.Close, timeperiod=n_days)

def get_macs(
    symbol_dataframe,
    fastperiod,
    fastmatype,
    slowperiod,
    slowmatype,
    signalperiod,
    signalmatype,
    ):
    return talib.MACDEXT(
        symbol_dataframe.Close, 
        fastperiod=fastperiod, 
        fastmatype=fastmatype, 
        slowperiod=slowperiod, 
        slowmatype=slowmatype, 
        signalperiod=signalperiod, 
        signalmatype=signalmatype,
        )

def get_stoch(
    symbol_dataframe,
    fastk_period, slowk_period, slowk_matype, slowd_period, slowd_matype
    ):
    return talib.STOCH(
        symbol_dataframe.High, symbol_dataframe.Low, symbol_dataframe.Close,
        fastk_period=fastk_period, 
        slowk_period=slowk_period, 
        slowk_matype=slowk_matype, 
        slowd_period=slowd_period, 
        slowd_matype=slowd_matype,
        )

def get_sar(symbol_dataframe, acceleration, maximum):
    return talib.SAR(
        symbol_dataframe['High'],
        symbol_dataframe['Low'],
        acceleration=acceleration, 
        maximum=maximum,
        )
    
def get_bbands(symbol_dataframe, timeperiod, nbdevup, nbdevdn, matype):
    return talib.BBANDS(
        symbol_dataframe.Close, 
        timeperiod=timeperiod, 
        nbdevup=nbdevup, 
        nbdevdn=nbdevdn, 
        matype=matype,
        )

def get_obv(symbol_dataframe):
    return talib.OBV(
        symbol_dataframe['Close'],
        symbol_dataframe['Volume'],
    )    

def get_adx(symbol_dataframe, timeperiod):
    return talib.ADX(
        symbol_dataframe['High'],
        symbol_dataframe['Low'],
        symbol_dataframe['Close'],
        timeperiod=timeperiod,
    )
    
def get_atr(symbol_dataframe, timeperiod):
    return talib.ATR(
        symbol_dataframe['High'],
        symbol_dataframe['Low'],
        symbol_dataframe['Close'],
        timeperiod=timeperiod,
    )

def generate_signal(symbol_dataframe, current_date):
    # calculating price
    symbol_dataframe['price'] = (symbol_dataframe.Open + symbol_dataframe.Close) / 2

    prepare_data(symbol_dataframe)
    
    row = symbol_dataframe.iloc[-2]
    prev_row = symbol_dataframe.iloc[-3]
    current_row = symbol_dataframe.iloc[-1]
    
    sar_score = gen_sar_score(row)
    macd_score = gen_macd_score(row)
    stoch_score = gen_stoch_score(row)
    bb_score = gen_bb_score(row, prev_row, current_row)

    cumulative_score = sar_score \
        + macd_score \
            + stoch_score \
                + bb_score
    
    if (cumulative_score >= 200) \
        and (current_row['Close'] > row['Close']) \
            and (current_row['rsi'] > 20) \
                and (current_row['rsi'] < 90):
        if (row['adx'] > 20):
            #print("BB Signal !!")
            return 'buy'
    if (row['EMA5_prev'] < row['EMA25_prev']) and (row['EMA5'] > row['EMA25']):
        # print("In EMA 5-25")
        if (row['rsi'] > 30) and (row['rsi'] < 90) and (row['adx'] > 20):
            return 'buy'
        else:
            return 'neutral'
    
    if (row['EMA50_prev'] < row['EMA200_prev']) and (row['EMA50'] > row['EMA200']):
        # print("In EMA 50-200")
        if (row['rsi'] > 30) and (row['rsi'] < 90) and (row['adx'] > 20):
            return 'buy'
        else:
            return 'neutral'
    elif (row['EMA50_prev'] > row['EMA200_prev']) and (row['EMA50'] < row['EMA200']):
        return 'sell'

    return 'neutral'

def gen_sar_score(row):
    if((row['sar_prev'] > row['close_prev']) and (row['sar'] < row['Close'])):
        return 200
    elif ((row['sar_prev'] < row['close_prev']) and (row['sar'] > row['Close'])):
        return -200
    else:
        return 0

def gen_macd_score(row):
    if((row['macdhist_prev'] < 0) and (row['macdhist'] > 0)):
        return 100
    elif ((row['macdhist_prev'] > 0) and (row['macdhist'] < 0)):
        return -100
    else:
        return 0

def gen_stoch_score(row):
    if((row['slowk_prev'] < row['slowd_prev']) and (row['slowk'] > row['slowd'])):
        return 100
    elif ((row['slowk_prev'] > row['slowd_prev']) and (row['slowk'] < row['slowd'])):
        return -100
    else:
        return 0

def gen_bb_score(row, prev_row, current_row):
    if (prev_row['Close'] < prev_row['lowerband']) and (row['Close'] > row['lowerband']) and (current_row['Close'] >= row['Close']):
        return 200
    else:
        return 0        

In [174]:
# Preview
df_extended = prepare_data(df_extended)
df_extended

,Open,High,Low,Close,Volume,symbol,updated_at,close_prev,EMA5,EMA5_prev,...,slowk,slowk_prev,slowd,slowd_prev,upperband,middleband,lowerband,obv,adx,atr
timestamp,,,,,,,,,,,,,,,,,,,,,
2018-08-20,1037.5,1053.80,1037.5000,1048.1000,3905568,HDFCBANK,2020-04-17 03:06:36,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3905568.0,NaN,NaN
2018-08-21,1049.0,1056.00,1046.3199,1048.0300,3888160,HDFCBANK,2020-04-17 03:06:36,1048.1000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17408.0,NaN,NaN
2018-08-23,1052.5,1053.50,1038.4301,1039.9500,6287276,HDFCBANK,2020-04-17 03:06:36,1048.0300,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6269868.0,NaN,NaN
2018-08-24,1035.5,1046.80,1033.0300,1034.9301,4857314,HDFCBANK,2020-04-17 03:06:36,1039.9500,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-11127182.0,NaN,NaN
2018-08-27,1038.5,1044.25,1035.1200,1040.2200,6287394,HDFCBANK,2020-04-17 03:06:36,1034.9301,1042.246020,NaN,...,NaN,NaN,NaN,NaN,1052.468452,1042.24602,1032.023588,-4839788.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-26,857.0,871.75,848.7500,852.4000,19002589,HDFCBANK,2020-08-06 06:13:25,838.8500,853.347911,853.821867,...,18.048295,16.857955,18.048295,16.857955,869.979207,847.71000,825.440793,-74630647.0,16.814008,42.697681
2020-05-27,859.9,909.80,857.1500,903.6500,28013227,HDFCBANK,2020-08-06 06:13:25,852.4000,870.115274,853.347911,...,29.026637,18.048295,29.026637,18.048295,906.057795,862.31000,818.562205,-46617420.0,16.096661,43.747847
2020-05-28,920.0,950.00,913.9500,945.2500,37744926,HDFCBANK,2020-08-06 06:13:25,903.6500,895.160183,870.115274,...,55.514065,29.026637,55.514065,29.026637,958.375461,879.94000,801.504539,-8872494.0,15.406526,43.933715


In [175]:
signal = generate_signal(df_extended, '2019-01-01')
signal

'neutral'

In [176]:
# Generate CSV
df_extended.to_csv('CSV/{}_{}_{}_indicators.csv'.format(
    request_dict['symbol'],
    request_dict['dates']['start'],
    request_dict['dates']['end'],
))

In [177]:
n_days_lst = [5, 10, 15]
p_target = 0.10
p_stoploss = -0.10

In [178]:
def get_true_signal(change_n):
    if change_n >= p_target:
        return 'BUY'
    elif change_n <= p_stoploss:
        return 'SELL'
    else:
        return 'NEUTRAL'

def get_generated_signal(current_date):
    return generate_signal(df_extended, current_date).upper()

In [179]:
df_close_plus = df.copy(deep=True).drop(columns=['updated_at'])
df_close_plus['calc_signal'] = df_close_plus.index.to_series().apply(get_generated_signal)
for n_days in n_days_lst:
    df_close_plus['Close_{}'.format(n_days)] = df_close_plus['Close'].shift(-n_days)
    df_close_plus['Change_{}'.format(n_days)] = (
        (df_close_plus['Close_{}'.format(n_days)] - df_close_plus['Close'])
    )/df_close_plus['Close']


    df_close_plus['Signal_{}'.format(n_days)] = df_close_plus['Change_{}'.format(n_days)].apply(get_true_signal)

minimal_df = df_close_plus[['calc_signal', 'Signal_15']].copy()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(minimal_df)

,calc_signal,Signal_15
timestamp,,
2018-08-20,NEUTRAL,NEUTRAL
2018-08-21,NEUTRAL,NEUTRAL
2018-08-23,NEUTRAL,NEUTRAL
2018-08-24,NEUTRAL,NEUTRAL
2018-08-27,NEUTRAL,NEUTRAL
2018-08-28,NEUTRAL,NEUTRAL
2018-08-29,NEUTRAL,NEUTRAL
2018-08-30,NEUTRAL,NEUTRAL
2018-08-31,NEUTRAL,NEUTRAL


In [180]:
# todo:
# target, stoploss to be provided by the little one
# for those t and s and taking a range of ns, gen True values, check how close they are to Nudge values.
# this will tell us nothing

# calc obv for nudge values
# run backtest
# deploy on prod

# IRR
# stock selection